# Cost Surface Model
## Dory's Hike From Home to Whitewater State Park
### Version 02 for lab 03 with updated optimal path comparisons using different weights

### Set Up

In [5]:
#import modules for data manipulation

import arcpy
import requests
import os
import zipfile
import io
from arcpy.sa import *

In [1]:
#Set variables for the working directory and project geodatabase
proj_dir = r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01"
proj_gdb = r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\L03pt01.gdb"

#set arcpy environment, this is where this script will store the imported data
arcpy.env.workspace = proj_dir

#allow files to be over written
arcpy.env.overwriteOutput = True

#test
arcpy.env.workspace

'C:\\Users\\MrJDF\\Desktop\\Arc1-Projects\\Arc1_lab03\\L03pt01'

In [4]:
# set up fnction to unzip requests and save to the project's working directory
def unzip(input_zipped, working_dir):
    get_zipped = zipfile.ZipFile(
        io.BytesIO(
            input_zipped.content)
    )
    
    get_zipped.extractall(working_dir)

### Dory's Coordinates

In [6]:
#import coordinate table from local file and convert to point feature layer
dory_coords = r"C:\Users\MrJDF\Desktop\Lab2_Dory\Arc1Lab02DoryCostSurface\DoryStartEndXYs.csv"
#these coords are backward in my table, so backwards in my code! lazy me.
arcpy.management.XYTableToPoint(dory_coords, "dory_start_end", "y_coord", "x_coord") # optional params: ({z_field}, {coordinate_system})

<Result 'C:\\Users\\MrJDF\\Desktop\\Arc1-Projects\\Arc1_lab03\\L03pt01\\dory_start_end.shp'>

In [11]:
# create bounding geometry to use as clipping mask/ area extent

# create 2km Buffer around start and end points
arcpy.analysis.Buffer("dory_start_end", "dory_start_end_Buffer", "2000 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

#create geometry to encompass buffered points
arcpy.management.MinimumBoundingGeometry("dory_start_end_Buffer", "dory_aoi", "CIRCLE", "ALL", None, "NO_MBG_FIELDS")

<Result 'C:\\Users\\MrJDF\\Desktop\\Arc1-Projects\\Arc1_lab03\\L03pt01\\L03pt01.gdb\\dory_aoi'>

### Data Wrangling

In [5]:
# County boundaries
county_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_counties/shp_bdry_counties.zip'
county_post_request = requests.post(county_url)
# county_post_request

unzip(county_post_request, proj_dir)


In [6]:
# MN Landcover
landcover_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'
get_landcover = requests.post(landcover_url)


unzip(get_landcover, proj_dir)


# Water routes

water_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/water_strahler_stream_order/shp_water_strahler_stream_order.zip'
get_hydro = requests.post(water_url)

unzip(get_hydro, proj_dir)



# Elevation 

elevation_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'
get_elev = requests.post(elevation_url)

unzip(get_elev, proj_dir)


In [12]:
# Roads

roads_url = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_roads_mndot_tis/shp_trans_roads_mndot_tis.zip'
get_roads = requests.post(roads_url)

unzip(get_roads, proj_dir)

### Clip Inputs to Local Extent

In [14]:
# Clip input data to study area 
#make this into a tidy loop: come back to streamline after I get it working
#Uncomment last line of each to execute clip
#set parameters
landCover = 'NLCD_2019_Land_Cover.tif'
#r'C:\Users\MrJDF\Desktop\Lab2_Dory\Arc1Lab02DoryCostSurface\NLCD 2019 - Land Cover.lyr'
water = 'streams_with_strahler_stream_order.shp'
# #r'C:\Users\MrJDF\Desktop\Lab2_Dory\Arc1Lab02DoryCostSurface\streams_with_strahler_stream_order.shp'
elevation = 'elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m'
# #r'C:\Users\MrJDF\Desktop\Lab2_Dory\Arc1Lab02DoryCostSurface\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m'
roads = 'STREETS_LOAD.shp'
#r'C:\Users\MrJDF\Desktop\Lab2_Dory\Arc1Lab02DoryCostSurface\STREETS_LOAD.shp'
clip_features = 'dory_aoi'
#out_feature_class = r'C:\Users\MrJDF\Desktop\Lab2_Dory\Arc1Lab02DoryCostSurface\study_area_water.shp'

#land, must use clip here for raster input
in_raster = landCover
rectangle = clip_features
land_out_raster = 'aoi_lndCvr'
arcpy.management.Clip(in_raster, rectangle, land_out_raster) #, {in_template_dataset}, {nodata_value}, {clipping_geometry}, {maintain_clipping_extent})

# #water, pairwise clip for vector
in_features = water
water_out_feature_class = 'aoi_water'
arcpy.analysis.PairwiseClip(in_features, clip_features, water_out_feature_class) #optional params, {cluster_tolerance}

#Elevation, must use clip here for raster input
in_raster = elevation
rectangle = clip_features
dem_out_raster = 'aoi_dem'
arcpy.management.Clip(in_raster, rectangle, dem_out_raster)

#roads
in_features = roads
road_out_feature_class = 'aoi_roads.shp'
arcpy.analysis.PairwiseClip(in_features, clip_features, road_out_feature_class)


<Result 'C:\\Users\\MrJDF\\Desktop\\Arc1-Projects\\Arc1_lab03\\L03pt01\\aoi_roads.shp'>

### Calculate Slope from DEM
- use this as basis to reclassify in next step
- dory prefers an even path, flattest route...consider this in the weights, does she want flat over terrain considerations?


In [15]:
#Import new library for slope function and other raster calculations
#from arcpy.ia import * # moved to top

# Set the local variables
in_dem = 'aoi_dem'

# Execute the Slope function
dem_slope = Slope(in_dem)

# Save the output
dem_slope.save('aoi_dem_slope')

### Remap and Reclassify Raster data based on user preferences
- adjust scale to 1-10 for use in weighted surface
- low values are preferred surfaces for travel

In [17]:
# new import for reclassify tool!
#from arcpy.sa import *

# Slope reclass
# Set local variables

inRaster = 'dem_slope'
reclassField = "Value"
# Define the RemapValue Object 
remap = RemapRange([[0,3,1], [3, 6, 2], [6, 10, 3],[10, 15, 4], [15, 20, 5], [20, 25, 6],[25, 30, 7], [30, 35, 8], [35, 40, 9], [40,75,10]])

# Execute Reclassify
outReclassify = Reclassify(inRaster, reclassField, remap, "NODATA")

# Save the output 
outReclassify.save("slope_reclass.tif")

# Landcover reclass
# Set local variables

inRaster = 'aoi_lndCvr'
reclassField = "Value"
# Define the RemapValue Object 
remap = RemapRange([[11,12, 10], [21, 24, 7], [31, 31, 3],[41, 41, 1], [42, 42, 1], [43, 43, 1],[52, 52, 5], [71, 71, 2], [81,81,10], [82,82,10], [90,90,10], [95,95,10]])

# Execute Reclassify
outReclassify = Reclassify(inRaster, reclassField, remap, "NODATA")

# Save the output 
outReclassify.save("landCvr_reclass.tif")


RuntimeError: ERROR 010240: Could not save raster dataset to C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\landCvr_reclass with output format GRID.

### Add Roads and Hydro data to analysis
- Use if Dory will cross streams at bridges, no where else
- use road data to make hydro data low cost where they cross, 
- here we delete the river values entirely, where the intersect with roads and reclassify smaller streams as crossable, but the larger streams as uncrossable by doubling their values with the reclassify tool.


In [18]:
# Buffer roads and hydrology to create surfaces with dimension of 2 pixel values ie 60m and 4 ie 120
roads_buffer = arcpy.analysis.Buffer('aoi_roads', 'aoi_roads_buffer', '60 Meters')
hydro_buffer = arcpy.analysis.Buffer('aoi_water','aoi_water_buffer', '120 Meters')

#use buffered roads to erase hydro data, creating crossable paths for Dory in the next step, where we use the river pixels to multiply values by 0, effectively deleting them from the cost surface.
# set parameters
in_features = hydro_buffer
erase_features= roads_buffer
out_feature_class =  "water_minus_roads"

#execute erase function
arcpy.analysis.Erase(in_features, erase_features, out_feature_class)


<Result 'C:\\Users\\MrJDF\\Desktop\\Arc1-Projects\\Arc1_lab03\\L03pt01\\water_minus_roads.shp'>

In [23]:
# Merge the water minus roads dataset with the area of interest boundaries to identify any no value cells
arcpy.management.Merge("water_minus_roads;dory_aoi", r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\L03pt01.gdb\water_minus_roads_Merge", 'FW_ID "FW_ID" true true false 10 Long 0 10,Max,#,water_minus_roads,FW_ID,-1,-1;KITTLE_NBR "KITTLE_NBR" true true false 50 Text 0 0,First,#,water_minus_roads,KITTLE_NBR,0,50;KITTLE_NAM "KITTLE_NAM" true true false 50 Text 0 0,First,#,water_minus_roads,KITTLE_NAM,0,50;LENGTH_MI "LENGTH_MI" true true false 19 Double 0 0,First,#,water_minus_roads,LENGTH_MI,-1,-1;FIXED "FIXED" true true false 1 Text 0 0,First,#,water_minus_roads,FIXED,0,1;ENTIRE "ENTIRE" true true false 1 Text 0 0,First,#,water_minus_roads,ENTIRE,0,1;VALID "VALID" true true false 1 Text 0 0,First,#,water_minus_roads,VALID,0,1;LABEL "LABEL" true true false 200 Text 0 0,First,#,water_minus_roads,LABEL,0,200;EDITED_BY "EDITED_BY" true true false 16 Text 0 0,First,#,water_minus_roads,EDITED_BY,0,16;EDITED_DAT "EDITED_DAT" true true false 8 Date 0 0,First,#,water_minus_roads,EDITED_DAT,-1,-1;EDIT_COMME "EDIT_COMME" true true false 50 Text 0 0,First,#,water_minus_roads,EDIT_COMME,0,50;EVENT_TYPE "EVENT_TYPE" true true false 16 Text 0 0,First,#,water_minus_roads,EVENT_TYPE,0,16;PUBLISH_DA "PUBLISH_DA" true true false 8 Date 0 0,First,#,water_minus_roads,PUBLISH_DA,-1,-1;CONTENT_DA "CONTENT_DA" true true false 8 Date 0 0,First,#,water_minus_roads,CONTENT_DA,-1,-1;SO_ID "SO_ID" true true false 5 Long 0 5,First,#,water_minus_roads,SO_ID,-1,-1;SO_VALUE "SO_VALUE" true true false 5 Long 0 5,First,#,water_minus_roads,SO_VALUE,-1,-1;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,water_minus_roads,Shape_Leng,-1,-1;BUFF_DIST "BUFF_DIST" true true false 19 Double 0 0,First,#,water_minus_roads,BUFF_DIST,-1,-1;ORIG_FID "ORIG_FID" true true false 10 Long 0 10,First,#,water_minus_roads,ORIG_FID,-1,-1', "NO_SOURCE_INFO")

# Converting to raster
#arcpy.conversion.FeatureToRaster("water_minus_roads_rstr", 77.2406288442668)
arcpy.conversion.FeatureToRaster("water_minus_roads_Merge", "SO_VALUE", r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\L03pt01.gdb\water_minus_roads_rstr", 77.2406288442668)
# Reclassify the raster to fit the study area scale and change no data to zeros for raster calculator compatability
arcpy.ddd.Reclassify("water_minus_roads_rstr", "Value", "1 2;2 4;3 6;4 8;5 10;NODATA 1", "wtr_rclss", "DATA")

#note to self, final output: wtr_rclss
# this represents streams with bridges and the entire aoi with values

<Result 'C:\\Users\\MrJDF\\Desktop\\Arc1-Projects\\Arc1_lab03\\L03pt01\\wtr_rclss'>

### Cost Surfaces

In [11]:
# create weighted cost surface from input reclassified raster data using map algebra

# Set local variables
inRaster1 = "landCvr_reclass.tif"
inRaster2 = "slope_reclass.tif" 
inRaster3 = "wtr_rclss" 

# create weighted sum table object for input for the weighted sum raster calculation
WSumTableObj = WSTable([[inRaster1, "VALUE", 0.5], [inRaster2, "VALUE", 0.25], [inRaster3, "VALUE", 0.25 ]])

# Execute WeightedSum
outWeightedSum = WeightedSum(WSumTableObj)

# Save the output 
outWeightedSum.save("weighted_surface")



In [ ]:
# Weighted Overlay method for weighted cost surface number 2
#out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\landCvr_reclass.tif' 50 'Value' (1 1; 2 2; 3 3; 5 6; 7 8; 10 10; NODATA NODATA); 'C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\slope_reclass.tif' 25 'Value' (1 1; 2 2; 3 3; 4 4; 5 5; 6 6; 7 7; 8 8; 9 9; 10 10; NODATA NODATA); 'C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\wtr_rclss' 25 'VALUE' (1 1; 2 2; 4 4; 6 7; 8 9; 10 10; NODATA NODATA));1 10 1"); out_raster.save(r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\L03pt01.gdb\WtOvrly_01")

#out_raster = arcpy.sa.WeightedOverlay(r"('C:\Users\MrJDF\Desktop\Lab2_Dory\Arc1Lab02DoryCostSurface\slope_reclass.tif' 50 'Value' (1 1; 2 2; 3 3; 4 4; 5 6; 6 7; 7 8; 8 9; 9 10; 10 1; NODATA NODATA); 'C:\Users\MrJDF\Desktop\Lab2_Dory\Arc1Lab02DoryCostSurface\landCvr_reclass.tif' 25 'Value' (1 1; 2 2; 3 3; 5 6; 7 8; 10 1; NODATA NODATA); 'C:\Users\MrJDF\Desktop\Lab2_Dory\Arc1Lab02DoryCostSurface\Arc1Lab02DoryCostSurface.gdb\Reclass_hydro' 25 'Value' (0 1; 2 2; 4 4; 5 6; 6 7; 8 9; NODATA NODATA));1 10 1"); out_raster.save(r"C:\Users\MrJDF\Desktop\Lab2_Dory\Arc1Lab02DoryCostSurface\Arc1Lab02DoryCostSurface.gdb\Weighted_srfc_02")

### Calculate Cost Path

#### Path 1 weights
* Landcover: 50%
* Slope: 25%
* Streams: 25%

In [12]:
#Create seperate start point for distance cost input
arcpy.conversion.FeatureClassToFeatureClass("dory_start_end", r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\L03pt01.gdb", "dory_start", "name = 'start'", 'name "name" true true false 254 Text 0 0,First,#,dory_start_end,name,0,254;x_coord "x_coord" true true false 19 Double 0 0,First,#,dory_start_end,x_coord,-1,-1;y_coord "y_coord" true true false 19 Double 0 0,First,#,dory_start_end,y_coord,-1,-1;new_ID "new_ID" true true false 13 Float 0 0,First,#,dory_start_end,new_ID,-1,-1', '')

#Create seperate endpoint for backlink
arcpy.conversion.FeatureClassToFeatureClass("dory_start_end", r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\L03pt01.gdb", "dory_end", "name = 'end'", 'name "name" true true false 254 Text 0 0,First,#,dory_start_end,name,0,254;x_coord "x_coord" true true false 19 Double 0 0,First,#,dory_start_end,x_coord,-1,-1;y_coord "y_coord" true true false 19 Double 0 0,First,#,dory_start_end,y_coord,-1,-1;new_ID "new_ID" true true false 13 Float 0 0,First,#,dory_start_end,new_ID,-1,-1', '')

#buffer to give pointts dimensions, I buffered with a 30 m radius to give at least 4 pixels to the points
#buffer_pts = arcpy.analysis.Buffer("dory_start_end", "dory_start_end_Buffer02", "100 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")
#arcpy.analysis.Buffer("dory_start_end", "dory_start_end_Buffer", "2000 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")
#Convert buffered points to rasters
#arcpy.conversion.FeatureToRaster( buffer_pts, "Shape_Length", "stend_rstr")
#arcpy.conversion.FeatureToRaster("dory_start_end_Buffer02", "BUFF_DIST", r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\L03pt01.gdb\startEndRstr", 0.000301001567788859)
#Create cost Distance surface from weighted surface
out_distance_raster = arcpy.sa.CostDistance("dory_start", "weighted_surface", None, None, None, None, None, None, ''); out_distance_raster.save("costDist_01")
#out_distance_raster = arcpy.sa.CostDistance("dory_start_end_Buffer02", "WtOvrly_01", None, None, None, None, None, None, ''); out_distance_raster.save(r"C:\Users\MrJDF\Desktop\Arc1-Projects\Arc1_lab03\L03pt01\L03pt01.gdb\CostDist_02")

#define variables
# inSources = "dory_start"
# inCost = "weighted_surface.tif"

# # Execute EucDirections
# outDistAcc = DistanceAccumulation(inSources, inCost)

# # Save the output 
# outDistAcc.save("distAcc.tif")

#create Cost Back Link from Weighted Overlay (required as input for cost path tool)
out_backlink_raster = arcpy.sa.CostBackLink("dory_end", "weighted_surface", None, None, None, None, None, None, ''); out_backlink_raster.save("CostBacklink_01")

#create cost path (note, I couldn't get this to run with other inputs, using the buffered featureset worked somehow)
out_raster = arcpy.sa.CostPath("dory_start", out_distance_raster, out_backlink_raster, "BEST_SINGLE", "OBJECTID", "INPUT_RANGE"); out_raster.save("costPath_01")

#### Path 2 weights
* Landcover: 20%
* Slope: 30%
* Streams: 50%

In [19]:
# create weighted cost surface from input reclassified raster data using map algebra

# Set local variables
inRaster1 = "landCvr_reclass.tif"
inRaster2 = "slope_reclass.tif" 
inRaster3 = "wtr_rclss" 

# create weighted sum table object for input for the weighted sum raster calculation
WSumTableObj02 = WSTable([[inRaster1, "VALUE", 0.05], [inRaster2, "VALUE", 0.90], [inRaster3, "VALUE", 0.05 ]])

# Execute WeightedSum
outWeightedSum02 = WeightedSum(WSumTableObj02)

# Save the output 
outWeightedSum02.save("weighted_surface02")



In [22]:
#Create cost Distance surface from weighted surface 2
out_distance_raster = arcpy.sa.CostDistance("dory_start", "weighted_surface02", None, None, None, None, None, None, ''); out_distance_raster.save("costDist_02")

#create Cost Back Link from Weighted surface 2 (required as input for cost path tool)
out_backlink_raster = arcpy.sa.CostBackLink("dory_end", "weighted_surface02", None, None, None, None, None, None, ''); out_backlink_raster.save("CostBacklink_02")

#create cost path (note, I couldn't get this to run with other inputs, using the buffered featureset worked somehow)
out_raster = arcpy.sa.CostPath("dory_start", out_distance_raster, out_backlink_raster, "BEST_SINGLE", "OBJECTID", "INPUT_RANGE"); out_raster.save("costPath_022")

#### Path 3 weights
* Landcover: 10%
* Slope: 70%
* Streams: 20%

In [15]:
# create weighted cost surface from input reclassified raster data using map algebra

# Set local variables
inRaster1 = "landCvr_reclass.tif"
inRaster2 = "slope_reclass.tif" 
inRaster3 = "wtr_rclss" 

# create weighted sum table object for input for the weighted sum raster calculation
WSumTableObj03 = WSTable([[inRaster1, "VALUE", 0.10], [inRaster2, "VALUE", 0.70], [inRaster3, "VALUE", 0.20 ]])

# Execute WeightedSum
outWeightedSum03 = WeightedSum(WSumTableObj03)

# Save the output 
outWeightedSum03.save("weighted_surface03")



In [16]:
#Create cost Distance surface from weighted surface 2
out_distance_raster = arcpy.sa.CostDistance("dory_start", "weighted_surface03", None, None, None, None, None, None, ''); out_distance_raster.save("costDist_03")

#create Cost Back Link from Weighted surface 2 (required as input for cost path tool)
out_backlink_raster = arcpy.sa.CostBackLink("dory_end", "weighted_surface03", None, None, None, None, None, None, ''); out_backlink_raster.save("CostBacklink_03")

#create cost path (note, I couldn't get this to run with other inputs, using the buffered featureset worked somehow)
out_raster = arcpy.sa.CostPath("dory_start", out_distance_raster, out_backlink_raster, "BEST_SINGLE", "OBJECTID", "INPUT_RANGE"); out_raster.save("costPath_03")